In [13]:
import pandas as pd
import numpy as np
import json
import networkx as nx
import time
import heapq
import subprocess
from subprocess import Popen, PIPE, STDOUT
from sys import platform
import time
!chmod +x spike-dasm.exe

In [150]:
# Read JSON
with open('hello.out', 'r') as f:
    json_lines = f.readlines()

inst_jsons = []
for line in json_lines:
    try:
        inst_jsons.append(json.loads(line))
    except json.JSONDecodeError:
        pass

In [157]:
event_names = ["IF1", "IF2", "IBuf", "EX", "MEM", "DIV", "WB", "LLWB", "COM", "RET", "DIV WB"]
start_stage = "IF1"
end_stages = ["RET", "LLWB"]
event_types = ["bytes", "pc", "bytes", "inst_bytes", "bytes", "bytes", "bytes", "bytes", "bytes", "bytes", "bytes"]
event_to_datatype = {e:d for e, d in zip(event_names, event_types)}

In [158]:
def generate_data_array(jsons):
    dasm_input = ""
    for json in jsons:
        if event_to_datatype[json["event_name"]] == "inst_bytes":
            dasm_input += "DASM(" + json["data"] + ")|"
        else:
            dasm_input += json["data"] + "|"
    dasm_input = dasm_input[:-1]
    if platform == "darwin":
        p = Popen("./spike-dasm --isa=rv64gcv", stdout=PIPE, stdin=PIPE, stderr=PIPE, text=True, shell=True)
    else:
        p = Popen("./spike-dasm.exe --isa=rv64gcv", stdout=PIPE, stdin=PIPE, stderr=PIPE, text=True, shell=True)
    stdout_data = p.communicate(input=dasm_input)[0]
    insts = stdout_data.split("|")
    return np.array(insts)

In [159]:
inst_ids = np.array([inst_jsons[i]["id"] for i in range(len(inst_jsons))])
inst_cycle = np.array([inst_jsons[i]["cycle"].strip() for i in range(len(inst_jsons))])
inst_event = np.array([inst_jsons[i]["event_name"] for i in range(len(inst_jsons))])
data_field = generate_data_array(inst_jsons)
inst_parent = np.array([inst_jsons[i]["parents"] for i in range(len(inst_jsons))])
data = np.column_stack((inst_ids,inst_parent, inst_cycle, inst_event, data_field))
columns = ["inst_id", "parent_id", "cycle", "stage", "data"]
df = pd.DataFrame(data=data, columns=columns)

In [169]:
df[df["parent_id"] == "0x00000000000221bd"]

,inst_id,parent_id,cycle,stage,data
140044,0x00000001000221be,0x00000000000221bd,139710,IF2,0x0080000000


In [170]:
df[df["parent_id"] == "0x00000001000221be"]

,inst_id,parent_id,cycle,stage,data
140047,0x00000004000221bf,0x00000001000221be,139711,EX,"li ra, 0"
140048,0x00000002000221bf,0x00000001000221be,139711,IBuf,0x0080000002


In [160]:
global id
id = 0
def construct_graph(df):
    DG = nx.DiGraph()
    for row in df.itertuples():
        DG.add_node(row.inst_id, cycle=row.cycle, data=row.data, stage=row.stage)
        if row.parent_id != "None":
            DG.add_edge(row.parent_id, row.inst_id)
    return DG            
def construct_speculative_trace(G):
    paths = []
    id = 0
    for node in G:
        data = G.nodes[node]
        if G.in_degree(node) == 0: # root node
            new_paths = trace_down(G, node, [], [])
            paths.extend(new_paths)
    for path in paths:
        if path[-1][0] not in end_stages:
            path.append(("FLUSH", str(int(path[-1][1]) + 1), "None"))
        else:
            path.append(("KONNATA_RET", str(int(path[-1][1]) + 1), "None"))
        path.insert(0, (id, path[0][-1]))
    return paths

def trace_down(G, node, curr_path, paths):
    id = id + 1
    data = G.nodes[node]
    curr_path.append((data["stage"], data["cycle"], data["data"]))
    if G.out_degree(node) == 0: # terminal node
        paths.append(curr_path)
        return paths
    succs = list(DG.successors(node))
    if data["stage"] not in split_points and len(succs) > 1:
        inst_paths = [trace_down(G, n, [], [])[0] for n in succs]
        inst_paths = inst_paths[1] + inst_paths[0]
        inst_paths.sort(key=lambda x: x[1])
        paths.append(curr_path + inst_paths)
    for n in succs:
        if n == sucs[0]:
            paths.extend(trace_down(G, n, curr_path[:], []))
        else:
            paths.extend(trace_down(G, n, [("DEP", cycle, )], []))
    return paths

def construct_committed_trace(G):
    paths = []
    id = 0
    for node in G:
        data = G.nodes[node]
        if G.out_degree(node) == 0 and data["stage"] in end_stages: # committed leaf node
            new_path = trace_up(G, node)
            new_path.insert(0, (id, data["data"]))
            paths.append(new_path)
            id += 1
    return paths

def trace_up(G, node):
    path = []
    while node:
        data = G.nodes[node]
        path.insert(0, (data["stage"], data["cycle"], data["data"]))
        node = list(DG.predecessors(node))[0] if list(DG.predecessors(node)) else ""
    return path
    

In [161]:
class InstructionTracer:
    def __init__(self, df):
        self.id = 0
        self.G = nx.DiGraph()
        for row in df.itertuples():
            self.G.add_node(row.inst_id, cycle=row.cycle, data=row.data, stage=row.stage)
            if row.parent_id != "None":
                self.G.add_edge(row.parent_id, row.inst_id)

    def construct_speculative_trace(self):
        self.id = 0
        paths = []
        for node in self.G:
            data = self.G.nodes[node]
            if self.G.in_degree(node) == 0: # root node
                new_paths = self.trace_down(node, [self.id], [])
                self.id += 1
                paths.extend(new_paths)
        for path in paths:
            if path[-1][0] not in end_stages:
                path.append(("FLUSH", int(path[-1][1]) + 1, "None"))
            else:
                path.append(("KONNATA_RET", int(path[-1][1]) + 1, "None"))
        return paths

    def trace_down(self, node, curr_path, paths):
        data = self.G.nodes[node]
        curr_path.append((data["stage"], int(data["cycle"]), data["data"]))
        if self.G.out_degree(node) == 0: # terminal node
            paths.append(curr_path)
            return paths
        succs = list(self.G.successors(node))
        # if data["stage"] not in split_points and len(succs) > 1:
        #     inst_paths = [self.trace_down(n, [], [])[0] for n in succs]
        #     inst_paths = inst_paths[1] + inst_paths[0]
        #     inst_paths.sort(key=lambda x: x[1])
        #     paths.append(curr_path + inst_paths)
        for n in succs:
            if n == succs[0]:
                paths.extend(self.trace_down(n, curr_path[:], []))
            else:
                self.id += 1
                paths.extend(self.trace_down(n, [self.id, ("DEP", int(data["cycle"]), str(curr_path[0]))], []))
        return paths


In [162]:
tracer = InstructionTracer(df)

In [163]:
paths = tracer.construct_speculative_trace()

In [165]:
import heapq

def convert_to_kanata(threads, verbose=False):
    pq = []
    if not verbose:
        threads = list(filter(lambda x: x[-1][0] == 'KONNATA_RET', threads)) #Relies on the last element of inst list being RET
    for inst in threads:
        id = inst[0]
        for stage in inst[1:]:
            heapq.heappush(pq, ((int(stage[1])), (id, stage[2], stage[0]))) #Min heap of (cycle -> (unique_id, pc, pipeline stage))
            
    with open('hello.log', 'w') as file:
        file.write('Kanata    0004\n')
        cycle, (id, pc, stage) = heapq.heappop(pq)
        prev_cycle = cycle
        file.write(f'C=\t{cycle}\n')
        while pq:
            cycle_diff = cycle - prev_cycle
            if (cycle_diff > 0):
                file.write(f"C\t{cycle_diff}\n")
            if (stage == start_stage):
                file.write(f"I\t{id}\t{cycle}\t0\n")
                # file.write(f"L    {id}    0    {pc}\n")
            if (stage == 'KONNATA_RET'):
                file.write(f"R\t{id}\t{id}\t0\n")
            elif (stage == 'DEP'):
                # spawn new row and draw dependency between this row and next
                file.write(f"I\t{id}\t{cycle}\t0\n")
                file.write(f"W\t{id}\t{int(pc)}\t1\n")
            elif (stage == 'FLUSH'):
                file.write(f"R\t{id}\t{id}\t1\n")
            elif (event_to_datatype[stage] == "inst_bytes"):
                file.write(f"S\t{id}\t0\t{stage}\n")
                file.write(f"L\t{id}\t0\tDASM:{pc}\n")
            elif (event_to_datatype[stage] == "pc"):
                file.write(f"S\t{id}\t0\t{stage}\n")
                file.write(f"L\t{id}\t0\tPC:{pc} \n")
            else:
                file.write(f"S\t{id}\t0\t{stage}\n")
                file.write(f"L\t{id}\t1\tPC:{pc}\n")

            prev_cycle = cycle
            cycle, (id, pc, stage) = heapq.heappop(pq)

In [166]:
convert_to_kanata(paths, verbose=True)